In [150]:
!pip install translate

You should consider upgrading via the 'C:\Users\39349\anaconda3\python.exe -m pip install --upgrade pip' command.


In [154]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from translate import Translator as tr
rawItalianColumns = pd.read_excel("D:/PHD/Projects/data/report.xlsx",usecols="A", engine="openpyxl", sheet_name="Questionario_2781022573")
responseColumns = pd.read_excel("D:/PHD/Projects/data/report.xlsx",skiprows=12,nrows=1, engine="openpyxl", sheet_name="Risposte_2781022573")
responseColumns.drop("Unnamed: 0", axis = 1, inplace=True)
responseColumns = responseColumns.columns.to_list()
responseColumns = [responseColumns[x].replace(",", ".",) for x in range(len(responseColumns))]

responseColumnsValues = []

for i in range(len(rawItalianColumns)):
    try:
        if any(responseColumns[x] in rawItalianColumns.iloc[i, 0] for x in range(len(responseColumns))):
            responseColumnsValues.append(rawItalianColumns.iloc[i, 0])
    except TypeError:
        pass

In [157]:
translator = tr(to_lang="en",from_lang="it")
textInItalian = tr.git remote 

Init signature:
tr(
    to_lang,
    from_lang='en',
    provider=None,
    secret_access_key=None,
    region=None,
    **kwargs,
)
Docstring:      <no docstring>
Source:        
class Translator:
    def __init__(self, to_lang, from_lang='en', provider=None, secret_access_key=None, region=None, **kwargs):
        self.available_providers = list(PROVIDERS_CLASS.keys())
        self.from_lang = from_lang
        self.to_lang = to_lang
        if provider and provider not in self.available_providers:
            raise InvalidProviderError(
                'Provider class invalid. '
                'Please check providers list below: {!r}'.format(self.available_providers)
            )

        provider_class = PROVIDERS_CLASS.get(provider, DEFAULT_PROVIDER)

        self.provider = provider_class(
            from_lang=from_lang,
            to_lang=to_lang,
            secret_access_key=secret_access_key,
            region=region,
            **kwargs
        )

    def translate(self